# Autogen code executors

From the documentation about code executors:

"In AutoGen, a code executor is a component that takes input messages (e.g., those containing code blocks), performs execution, and outputs messages with the results. AutoGen provides two types of built-in code executors, one is command line code executor, which runs code in a command line environment such as a UNIX shell, and the other is Jupyter executor, which runs code in an interactive Jupyter kernel."

References:

- [Code executors](https://microsoft.github.io/autogen/docs/tutorial/code-executors/)

## Install the required packages

In [ ]:
%pip install -q python-dotenv==1.0.1 openai==1.35.9 gradio==4.39.0 pyautogen==0.2.32

## Import the required packages and create the Azure OpenAI client

In [2]:
import os
import pprint
from pathlib import Path
from autogen import ConversableAgent
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
from dotenv import load_dotenv

load_dotenv("../../.env")

model = os.getenv("GPT_MODEL")
endpoint=os.getenv("ENDPOINT")
api_key=os.getenv("API_KEY")
api_version=os.getenv("API_VERSION")

## Test the local code executor

- The executore is capable of executing code and returning the results

In [3]:
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code="print('Hello Word!')\nfor i in range(0,10,2):\n  print(i)"),
        ]
    )
)

exit_code=0 output='Hello Word!\n0\n2\n4\n6\n8\n' code_file='/home/alex/github/msalmor/ai-code-blocks/python/notebooks/autogen/coding/tmp_code_92936a0d0dd7418fea73f87db7c6ff35.py'


## Config an Autogen conversable agent

In [4]:
config_list = [
    {
        "base_url": endpoint,
        "api_key": api_key,
        "model": model,
        "api_type": "azure",
        "api_version": api_version
    }
]

llm_config = {
    "model": model,
    "temperature": 0,
    "config_list": config_list,    
    #"cache_seed": None, # Enable caching
}

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

## Generate a python script to find the 1001st prime number

In [6]:
reply = agent.generate_reply(messages=[{"role":"user","content":"Write a pythong script to find the 1001st prime number."}])
print(reply)

Certainly! To find the 1001st prime number, you can use a simple algorithm that checks for prime numbers and counts them until it reaches the 1001st prime. Here's a Python script to accomplish this:

```python
def is_prime(n):
    """Check if a number is a prime number."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def find_nth_prime(n):
    """Find the nth prime number."""
    count = 0
    num = 1
    while count < n:
        num += 1
        if is_prime(num):
            count += 1
    return num

# Find the 1001st prime number
nth_prime = 1001
prime_number = find_nth_prime(nth_prime)
print(f"The {nth_prime}st prime number is: {prime_number}")
```

This script defines two functions:
1. `is_prime(n)`: Checks if a number `n` is prime.
2. `find_nth_prime(n)`: Finds the `n`th 

## Extract the Python code, execute it, and print the result

In [9]:
# Using regex get the python code in the markdown
import re
code_groups = re.search(r'```python\n(.*?)```', reply, re.DOTALL)

if len(code_groups.groups()) > 0:
    code_execution_result = executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code=code_groups.group(1)),
            ]
        )

    print(code_execution_result)

exit_code=0 output='The 1001st prime number is: 7927\n' code_file='/home/alex/github/msalmor/ai-code-blocks/python/notebooks/autogen/coding/tmp_code_8de17a467e5ed350256ec0d4ef04c063.py'
